In [16]:
from datetime import datetime

from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket.utils.operators import QubitPauliOperator
from pytket.partition import measurement_reduction,MeasurementBitMap, MeasurementSetup, PauliPartitionStrat
from pytket.backends.backendresult import BackendResult

from pytket.utils.operators import QubitPauliOperator
from pytket.pauli import Pauli, QubitPauliString
from pytket.circuit import Qubit

from scipy.optimize import minimize
from numpy import ndarray
from numpy.random import random_sample
from sympy import Symbol

from qnexus.client import circuits, projects, jobs
from qnexus.references import CircuitRef, ProjectRef



# Example VQE workflow using Nexus


VQE example adapted from https://github.com/CQCL/pytket-quantinuum/blob/develop/examples/Quantinuum_variational_experiment_with_batching.ipynb

## Set up the VQE components

In [17]:
# 1. Synthesise Symbolic State-Preparation Circuit (hardware efficient ansatz)

symbols = [Symbol(f"p{i}") for i in range(4)]
symbolic_circuit = Circuit(2)
symbolic_circuit.X(0)
symbolic_circuit.Ry(symbols[0], 0).Ry(symbols[1], 1)
symbolic_circuit.CX(0, 1)
symbolic_circuit.Ry(symbols[2], 0).Ry(symbols[3], 0)

#render_circuit_jupyter(symbolic_circuit)

[X q[0]; Ry(p1) q[1]; Ry(p0) q[0]; CX q[0], q[1]; Ry(p2) q[0]; Ry(p3) q[0]; ]

In [18]:
# 2. Define Hamiltonian 
# coefficients in the Hamiltonian are obtained from PhysRevX.6.031007

coeffs = [-0.4804, 0.3435, -0.4347, 0.5716, 0.0910, 0.0910]
term0 = {
    QubitPauliString(
        {
            Qubit(0): Pauli.I,
            Qubit(1): Pauli.I,
        }
    ): coeffs[0]
}
term1 = {QubitPauliString({Qubit(0): Pauli.Z, Qubit(1): Pauli.I}): coeffs[1]}
term2 = {QubitPauliString({Qubit(0): Pauli.I, Qubit(1): Pauli.Z}): coeffs[2]}
term3 = {QubitPauliString({Qubit(0): Pauli.Z, Qubit(1): Pauli.Z}): coeffs[3]}
term4 = {QubitPauliString({Qubit(0): Pauli.X, Qubit(1): Pauli.X}): coeffs[4]}
term5 = {QubitPauliString({Qubit(0): Pauli.Y, Qubit(1): Pauli.Y}): coeffs[5]}
term_sum = {}
term_sum.update(term0)
term_sum.update(term1)
term_sum.update(term2)
term_sum.update(term3)
term_sum.update(term4)
term_sum.update(term5)
hamiltonian = QubitPauliOperator(term_sum)



In [19]:
# 3 Computing Expectation Values

# Computing Expectation Values for Pauli-Strings
def compute_expectation_paulistring(
    distribution: dict[tuple[int, ...], float], bitmap: MeasurementBitMap
) -> float:
    value = 0
    for bitstring, probability in distribution.items():
        value += probability * (sum(bitstring[i] for i in bitmap.bits) % 2)
    return ((-1) ** bitmap.invert) * (-2 * value + 1)



# 3.2 Computing Expectation Values for sums of Pauli-strings multiplied by coefficients
def compute_expectation_value(
    results: list[BackendResult],
    measurement_setup: MeasurementSetup,
    operator: QubitPauliOperator,
) -> float:
    energy = 0
    for pauli_string, bitmaps in measurement_setup.results.items():
        string_coeff = operator.get(pauli_string, 0.0)
        if string_coeff > 0:
            for bm in bitmaps:
                index = bm.circ_index
                distribution = results[index].get_distribution()
                value = compute_expectation_paulistring(distribution, bm)
                energy += complex(value * string_coeff).real
    return energy

In [21]:
# 4. Building our Objective function

class Objective:
    def __init__(
        self,
        nexus_project: ProjectRef,
        symbolic_circuit: CircuitRef,
        problem_hamiltonian: QubitPauliOperator,
        n_shots_per_circuit: int,
        iteration_number: int = 0,
        n_iterations: int = 10,
    ) -> None:
        r"""Returns the objective function needed for a variational
        procedure.
        """
        terms = [term for term in problem_hamiltonian._dict.keys()]
        self._symbolic_circuit: Circuit = symbolic_circuit.get_circuit()
        self._hamiltonian: QubitPauliOperator = problem_hamiltonian
        self._nshots: int = n_shots_per_circuit
        self._measurement_setup: MeasurementSetup = measurement_reduction(
            terms, strat=PauliPartitionStrat.CommutingSets
        )
        self._nexus_project = nexus_project
        self._iteration_number: int = iteration_number
        self._niters: int = n_iterations

    def __call__(self, parameter: ndarray) -> float:
        value = self._objective_function(parameter)
        self._iteration_number += 1
        if self._iteration_number >= self._niters:
            self._iteration_number = 0
        return value
    
    def _objective_function(
        self,
        parameters: ndarray,
    ) -> float:
        assert len(parameters) == len(self._symbolic_circuit.free_symbols())
        circuit_list = self._build_circuits(parameters)
        
        # Execute circuits with Nexus
        execute_job_ref =jobs.submit_execute_job(
            name=f"execute_job_VQE_{datetime.now()}_{self._iteration_number}",
            circuits=circuit_list,
            project=self._nexus_project,
            n_shots=[self._nshots]*len(circuit_list)
        )
        jobs.wait_for(execute_job_ref)
        results = [item.get_result() for item in jobs.execution_results(execute_job_ref)]

        expval = compute_expectation_value(
            results, self._measurement_setup, self._hamiltonian
        )
        return expval
    
    def _build_circuits(self, parameters: ndarray) -> list[CircuitRef]:
        circuit = self._symbolic_circuit.copy()
        symbol_dict = {s: p for s, p in zip(self._symbolic_circuit.free_symbols(), parameters)}
        circuit.symbol_substitution(symbol_dict)

        properties = {str(sym): val for sym, val in symbol_dict.items()}
        properties["iteration"] = self._iteration_number

        # Upload the numerical state-prep circuit to Nexus
        circuits.submit(
            circuit=circuit, 
            project=self._nexus_project,
            name=f"state prep circuit {self._iteration_number}",
            description="A numerical state-prep circuit for my dihydrogen VQE",
            properties=properties
        )
        circuit_list = []
        for mc in self._measurement_setup.measurement_circs:
            c = circuit.copy()
            c.append(mc)
            # Upload each measurement circuit to Nexus with correct params
            measurement_circuit_ref = circuits.submit(
                circuit=c, 
                project=self._nexus_project,
                name=f"state prep circuit {self._iteration_number}",
                description="A numerical state-prep circuit for my dihydrogen VQE",
                properties=properties
            )
            circuit_list.append(measurement_circuit_ref)
        # Compile circuits with Nexus
        compile_job_ref = jobs.submit_compile_job(
            name=f"compile_job_VQE_{datetime.now()}_{self._iteration_number}",
            circuits=circuit_list,
            optimisation_level=2,
            project=self._nexus_project,
        )
        jobs.wait_for(compile_job_ref)
        compiled_circuit_refs = jobs.compilation_results(compile_job_ref)

        for ref in compiled_circuit_refs:
            circuits.update(
                ref, 
                name=ref.annotations.name, 
                description=ref.annotations.description,
                properties=properties
            )
        return compiled_circuit_refs



## Set up the Nexus Project and run the VQE

In [22]:
# set up the project
project_ref = projects.submit(
    name=f"VQE_example_{str(datetime.now())}",
    description="A VQE done with qnexus",
    properties={}
)

In [23]:
# Set up the properties
projects.add_property(
    project_ref, 
    name="iteration", 
    property_type="int", 
    description=f"The iteration number in my dihydrogen VQE experiment", 
    required=False
)

for sym in symbolic_circuit.free_symbols():
    sym_name = str(sym)
    projects.add_property(
        project_ref, 
        name=sym_name, 
        property_type="float", 
        description=f"Our VQE {sym_name} parameter", 
        required=False
    )

In [24]:
# Upload our ansatz circuit

ansatz_ref = circuits.submit(
    circuit=symbolic_circuit,
    project=project_ref,
    name="ansatz_circuit",
    description="The ansatz state-prep circuit for my dihydrogen VQE",
    properties={}
)

In [25]:
objective = Objective(
    project_ref,
    ansatz_ref,
    hamiltonian,
    n_shots_per_circuit = 500,
    n_iterations= 4,
)

In [26]:
initial_parameters = random_sample(len(symbolic_circuit.free_symbols()))

result = minimize(
    objective,
    initial_parameters,
    method="COBYLA",
    options={"disp": True, "maxiter": objective._niters},
    tol=1e-2,
)

print(result.fun)
print(result.x)

-0.1178357421875

[1.64604383 0.80104275 0.93370373 0.53358902]
   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =    4   F =-1.178357E-01    MAXCV = 0.000000E+00
   X = 1.646044E+00   8.010427E-01   9.337037E-01   5.335890E-01


# Use Nexus to Rescue a VQE workflow

For instance, lets say that some failure happened on the 2nd iteration (e.g. laptop ran out of battery) and we want to resume ASAP.

In the above we ran for 4 iterations, lets pretend that we actually wanted to run for 7 and it failed on the 4th one.

In [28]:
# Get the project
project_ref = projects.projects(name="VQE_example_2024-01-11 16:44:56.065265")[-1]

project_ref

ProjectRef(id=UUID('f1b00222-d93e-4835-8042-58c886f12b40'), annotations=Annotations(name='VQE_example_2024-01-11 16:44:56.065265', description='A VQE done with qnexus', properties={}))

In [29]:
# Get the symbolic circuit

from qnexus.client import circuits

symb_circuits = circuits.circuits(name="ansatz_circuit", project_id=str(project_ref.id))

symbolic_circuit_ref = symb_circuits[0]

In [34]:
most_recent_circuits = circuits.circuits(name="final", project_id=project_ref.id)
for c in most_recent_circuits:
    print(c.annotations.properties["iteration"])

0
0
1
1
2
2
3
3


In [31]:
# Get the latest circuit to get the 'initial_parameters'
latest_circuit: CircuitRef = most_recent_circuits[-1]
print(latest_circuit)


latest_circuit_properties = latest_circuit.annotations.properties
print(latest_circuit_properties)

id=UUID('adb4370b-5f0b-49f3-9185-1400d5d6b7e1') annotations=Annotations(name='state prep circuit 3-AerBackend-final', description=None, properties={'p1': 1.9337037801742554, 'p2': 0.8010427355766296, 'p3': 1.6460437774658203, 'iteration': 3, 'p0': 0.5335890054702759}) project=ProjectRef(id=UUID('f1b00222-d93e-4835-8042-58c886f12b40'), annotations=Annotations(name='VQE_example_2024-01-11 16:44:56.065265', description='A VQE done with qnexus', properties={}))
{'p1': 1.9337037801742554, 'p2': 0.8010427355766296, 'p3': 1.6460437774658203, 'iteration': 3, 'p0': 0.5335890054702759}


In [32]:
# Get what iteration we were on (from the latest circuit)

last_iteration_count = latest_circuit_properties.pop("iteration")
print(last_iteration_count)


3


In [12]:
# Retreive the params and check them

print(latest_circuit_properties)

sorted_symbols = sorted([str(sym) for sym in symbolic_circuit_ref.get_circuit().free_symbols()])

new_starting_params = [latest_circuit_properties[str(symbol)] for symbol in sorted_symbols]

print(new_starting_params)


{'p0': 0.2365649938583374, 'p3': 1.436293363571167, 'p2': 0.8855235576629639, 'p1': 0.13932426273822784}
[0.2365649938583374, 0.13932426273822784, 0.8855235576629639, 1.436293363571167]


In [13]:
# Build the Objective and run 'minimize' to continue the experiment
objective = Objective(
    project_ref,
    symbolic_circuit_ref,
    hamiltonian,
    n_shots_per_circuit = 500,
    iteration_number=last_iteration_count, # resume from 3rd iteration of 7
    n_iterations = 7,
)

result = minimize(
    objective,
    new_starting_params,
    method="COBYLA",
    options={"disp": True, "maxiter": objective._niters},
    tol=1e-2,
)

print(result.fun)
print(result.x)

-0.1557009765625
   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =    3   F =-1.557010E-01    MAXCV = 0.000000E+00
   X = 2.365650E-01   1.393243E-01   8.855236E-01   1.436293E+00

[0.23656499 0.13932426 0.88552356 1.43629336]


In [ ]:
# TODO double check iteration prop is correct